In [108]:
import torch
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Flatten
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Reshape


In [109]:
plt.rc('font', family='AppleGothic')

In [110]:
train = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/train.csv')
trade = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/international_trade.csv')
test = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/test.csv')

In [111]:
train = train.rename(columns={'supply(kg)': 'supply', 'price(원/kg)': 'price'})
train.drop(columns=['ID'], inplace=True) 
test.drop(columns=['ID'], inplace=True)

### 파생변수 1: year, month, day, weekday

In [118]:
#연월일
train['year'] = pd.to_datetime(train['timestamp']).dt.year
train['month'] = pd.to_datetime(train['timestamp']).dt.month
train['day'] = pd.to_datetime(train['timestamp']).dt.day
train['weekday'] = train['timestamp'].dt.dayofweek

test['year'] = pd.to_datetime(test['timestamp']).dt.year
test['month'] = pd.to_datetime(test['timestamp']).dt.month
test['day'] = pd.to_datetime(test['timestamp']).dt.day
train['weekday'] = train['timestamp'].dt.dayofweek


### 파생변수 2 : 계절

12 - 2 / 
3 - 5 /
6 - 8 /
9 - 11

In [113]:
def map_season(month):
    if month in [12, 1, 2]:
        return 'win'
    elif month in [3, 4, 5]:
        return 'spr'
    elif month in [6, 7, 8]:
        return 'sum'
    elif month in [9, 10, 11]:
        return 'aut'
    
train['season'] = train['month'].apply(map_season)
test['season'] = test['month'].apply(map_season)

### price, supply 0값 처리
- idea 1. 

    모두 해당 year - month의 item의 price, supply 평균으로 대체

- idea 2.

    train에서 전날/다음날 같은 아이템의 price,supply 데이터가 있다면 똑같은 값으로 채우고 없다면 그 month의 item의 price,supply 평균으로 대체

#### idea 1 적용 MSE: 640037.0224936566

In [114]:
'''
# 'price'와 'supply'가 0인 행을 해당 'item' 종목 및 'month' 별 평균값으로 대체
for index, row in train[(train['price'] == 0)].iterrows():
    item = row['item']
    year = row['year']
    month = row['month']
    
    # 해당 'item' 및 'month'에 대한 평균값 계산
    avg_price = train[(train['item'] == item) & (train['year'] == year)& (train['month'] == month)]['price'].mean()
    avg_supply = train[(train['item'] == item) & (train['year'] == year) & (train['month'] == month)]['supply'].mean()
    
    # 대체
    train.at[index, 'price'] = avg_price
    train.at[index, 'supply'] = avg_supply

# 결과 확인
print(train.head())
'''

"\n# 'price'와 'supply'가 0인 행을 해당 'item' 종목 및 'month' 별 평균값으로 대체\nfor index, row in train[(train['price'] == 0)].iterrows():\n    item = row['item']\n    year = row['year']\n    month = row['month']\n    \n    # 해당 'item' 및 'month'에 대한 평균값 계산\n    avg_price = train[(train['item'] == item) & (train['year'] == year)& (train['month'] == month)]['price'].mean()\n    avg_supply = train[(train['item'] == item) & (train['year'] == year) & (train['month'] == month)]['supply'].mean()\n    \n    # 대체\n    train.at[index, 'price'] = avg_price\n    train.at[index, 'supply'] = avg_supply\n\n# 결과 확인\nprint(train.head())\n"

#### idea 2 적용 MSE: 606570.1850909404

In [116]:
import pandas as pd

# train['timestamp'] 열을 datetime 형식으로 변환
train['timestamp'] = pd.to_datetime(train['timestamp'])

# 0 값을 처리하는 함수 정의
def fill_zero_values(row):
    if row['price'] == 0 or row['supply'] == 0:
        # 같은 연도의 전날 데이터가 있는 경우
        prev_day_data = train[(train['timestamp'] == row['timestamp'] - pd.Timedelta(days=1)) & (train['item'] == row['item'])]
        next_day_data = train[(train['timestamp'] == row['timestamp'] + pd.Timedelta(days=1)) & (train['item'] == row['item'])]

        if not prev_day_data.empty and prev_day_data.shape[0] > 0 and prev_day_data['price'].values[0] != 0 and prev_day_data['supply'].values[0] != 0:
            row['price'] = prev_day_data['price'].values[0]
            row['supply'] = prev_day_data['supply'].values[0]
        elif not next_day_data.empty and next_day_data.shape[0] > 0 and next_day_data['price'].values[0] != 0 and next_day_data['supply'].values[0] != 0:
            # 같은 연도의 다음날 데이터가 있는 경우
            row['price'] = next_day_data['price'].values[0]
            row['supply'] = next_day_data['supply'].values[0]
        else:
            # 해당 'item' 및 'month'에 대한 평균값 계산
            avg_price = train[(train['item'] == row['item']) & (train['year'] == row['year']) & (train['month'] == row['month'])]['price'].mean()
            avg_supply = train[(train['item'] == row['item']) & (train['year'] == row['year']) & (train['month'] == row['month'])]['supply'].mean()

            # 대체
            row['price'] = avg_price if avg_price != 0 else 0
            row['supply'] = avg_supply if avg_supply != 0 else 0
    return row

# 0 값을 처리하는 함수를 적용
train = train.apply(fill_zero_values, axis=1)

# 결과 확인
print(train.head())


   timestamp item corporation location        supply        price  year  \
0 2019-01-01   TG           A        J  47433.957097  1485.383871  2019   
1 2019-01-02   TG           A        J  60601.000000  1728.000000  2019   
2 2019-01-03   TG           A        J  60601.000000  1728.000000  2019   
3 2019-01-04   TG           A        J  25000.000000  1408.000000  2019   
4 2019-01-05   TG           A        J  32352.000000  1250.000000  2019   

   month  day season  
0      1    1    win  
1      1    2    win  
2      1    3    win  
3      1    4    win  
4      1    5    win  


### 파생변수 3 : 해당 연도의 월별 item별 price의 평균

In [124]:
average_price_by_month_item = train.groupby(['year', 'month', 'item'])['price'].mean().reset_index()
train = pd.merge(train, average_price_by_month_item, on=['year', 'month', 'item'], how='left', suffixes=('', '_monavg'))
print(train.head())


   timestamp item corporation location        supply        price  year  \
0 2019-01-01   TG           A        J  47433.957097  1485.383871  2019   
1 2019-01-02   TG           A        J  60601.000000  1728.000000  2019   
2 2019-01-03   TG           A        J  60601.000000  1728.000000  2019   
3 2019-01-04   TG           A        J  25000.000000  1408.000000  2019   
4 2019-01-05   TG           A        J  32352.000000  1250.000000  2019   

   month  day season  weekday  price_monavg  
0      1    1    win        1   1801.396254  
1      1    2    win        2   1801.396254  
2      1    3    win        3   1801.396254  
3      1    4    win        4   1801.396254  
4      1    5    win        5   1801.396254  


### 파생변수 4 : 해당 연도의 계절별 item별 price의 평균

In [125]:
# 'year', 'month', 'item' 별로 그룹화하여 'price'의 평균 계산
average_price_by_year_month_item = train.groupby(['year', 'season', 'item'])['price'].mean().reset_index()

# 'train' 데이터프레임에 'year', 'month', 'item'에 대한 평균 'price' 값을 추가
train = pd.merge(train, average_price_by_year_month_item, on=['year', 'season', 'item'], how='left', suffixes=('', '_seasonavg'))

# 결과 확인
print(train.head())

   timestamp item corporation location        supply        price  year  \
0 2019-01-01   TG           A        J  47433.957097  1485.383871  2019   
1 2019-01-02   TG           A        J  60601.000000  1728.000000  2019   
2 2019-01-03   TG           A        J  60601.000000  1728.000000  2019   
3 2019-01-04   TG           A        J  25000.000000  1408.000000  2019   
4 2019-01-05   TG           A        J  32352.000000  1250.000000  2019   

   month  day season  weekday  price_monavg  price_seasonavg  
0      1    1    win        1   1801.396254      1828.194345  
1      1    2    win        2   1801.396254      1828.194345  
2      1    3    win        3   1801.396254      1828.194345  
3      1    4    win        4   1801.396254      1828.194345  
4      1    5    win        5   1801.396254      1828.194345  


### 파생변수 5. 타임스탬프 -> 365일 순환

In [126]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
def cyclic_encode_day_of_year(df, column='timestamp'):
    df['day_of_year'] = df[column].dt.dayofyear
    df['sin_day'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['cos_day'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    df.drop(['day_of_year'], axis=1, inplace=True)
    return df

train = cyclic_encode_day_of_year(train)

print(train.head())

   timestamp item corporation location        supply        price  year  \
0 2019-01-01   TG           A        J  47433.957097  1485.383871  2019   
1 2019-01-02   TG           A        J  60601.000000  1728.000000  2019   
2 2019-01-03   TG           A        J  60601.000000  1728.000000  2019   
3 2019-01-04   TG           A        J  25000.000000  1408.000000  2019   
4 2019-01-05   TG           A        J  32352.000000  1250.000000  2019   

   month  day season  weekday  price_monavg  price_seasonavg   sin_day  \
0      1    1    win        1   1801.396254      1828.194345  0.017213   
1      1    2    win        2   1801.396254      1828.194345  0.034422   
2      1    3    win        3   1801.396254      1828.194345  0.051620   
3      1    4    win        4   1801.396254      1828.194345  0.068802   
4      1    5    win        5   1801.396254      1828.194345  0.085965   

    cos_day  
0  0.999852  
1  0.999407  
2  0.998667  
3  0.997630  
4  0.996298  


In [28]:
#train의 timestamp drop
#train.drop(['timestamp'], axis=1, inplace=True)

### Trade 데이터 활용 (보류)

월별 item 값 구해서 train, test에 해당 값 넣어주기 
이건 상관관계 보고 수출금액 수입금액 무역수지 뭘쓸지 고민해야 할 듯 

In [24]:
trade.head()

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38


### test에서 supply 활용

- idea 1.

    train에서의 아이템별, 월별 평균으로 test에 supply 열 만들기
- idea 2.

    supply drop ..? 최후의 수단

In [ ]:
'''
# 'train' 데이터프레임에 'month' 및 'item' 별 'supply' 평균 계산
mean_supply_by_month_item = train.groupby(['month', 'item'])['supply'].mean()

# 'test' 데이터프레임에 'month' 및 'item'을 기준으로 'supply' 평균값을 병합 (merge)
test_merged = pd.merge(test, mean_supply_by_month_item.reset_index(), on=['month', 'item'], how='left')

# 'supply_y' 열은 평균값을 의미하므로, 'supply_y' 열을 'supply'로 이름 변경
test_merged.rename(columns={'supply_y': 'supply'}, inplace=True)

# 'test' 데이터프레임에 적용된 수정 사항을 확인
test = test_merged
'''

### dummy 변환

In [127]:
train = pd.get_dummies(train, columns=['item', 'corporation', 'location'])

'''
test.drop(columns=['timestamp'], inplace=True)
df_encoded2 = pd.get_dummies(test, columns=['item','corporation', 'location'])
test= df_encoded2
'''


"\ntest.drop(columns=['timestamp'], inplace=True)\ndf_encoded2 = pd.get_dummies(test, columns=['item','corporation', 'location'])\ntest= df_encoded2\n"

In [128]:
train.columns

Index(['timestamp', 'supply', 'price', 'year', 'month', 'day', 'season',
       'weekday', 'price_monavg', 'price_seasonavg', 'sin_day', 'cos_day',
       'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'corporation_F', 'location_J', 'location_S'],
      dtype='object')

In [129]:
from sklearn.model_selection import train_test_split


# 날짜 기반으로 데이터 정렬하여 끝의 30일을 test set으로 만듦
train = train.sort_values(by='timestamp')

# 날짜를 기준으로 그룹화
grouped_data = train.groupby('timestamp')

# 각 그룹에서 마지막 30일을 테스트 세트로 선택
test_data = pd.concat([group.tail(30) for name, group in grouped_data])

# 나머지를 훈련 세트로 선택
train_data = train.drop(test_data.index)

# 훈련 데이터와 테스트 데이터를 나누기
X_train = train_data[['supply', 'weekday','price_monavg',
       'price_seasonavg', 'sin_day', 'cos_day', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'corporation_F',
       'location_J', 'location_S']]
y_train = train_data['price']

X_test = test_data[['supply','weekday', 'price_monavg',
       'price_seasonavg', 'sin_day', 'cos_day', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'corporation_F',
       'location_J', 'location_S']]
y_test = test_data['price']

# 결과 확인
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (13707, 19)
y_train shape: (13707,)
X_test shape: (45690, 19)
y_test shape: (45690,)


In [130]:
y_train = np.reshape(y_train, (len(y_train), 1))
y_test = np.reshape(y_test, (len(y_test), 1))

In [131]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# 데이터 정규화
scaler_X = MinMaxScaler()
normalized_X_train = scaler_X.fit_transform(X_train)

scaler_y = MinMaxScaler()
normalized_y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))

# LSTM에 입력으로 넣기 위한 형태로 변환
# (샘플 수, 타임 스텝 수, 특성 수)
# 여기서는 간단하게 타임 스텝을 1로 설정합니다.
X_train_reshaped = normalized_X_train.reshape((normalized_X_train.shape[0], 1, normalized_X_train.shape[1]))

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(X_train_reshaped, normalized_y_train, epochs=70, batch_size=32, shuffle=False)



Epoch 1/70
429/429 [==============================] - 1s 628us/step - loss: 0.0055
Epoch 2/70
429/429 [==============================] - 0s 592us/step - loss: 0.0032
Epoch 3/70
429/429 [==============================] - 0s 591us/step - loss: 0.0032
Epoch 4/70
429/429 [==============================] - 0s 590us/step - loss: 0.0034
Epoch 5/70
429/429 [==============================] - 0s 594us/step - loss: 0.0034
Epoch 6/70
429/429 [==============================] - 0s 592us/step - loss: 0.0041
Epoch 7/70
429/429 [==============================] - 0s 589us/step - loss: 0.0039
Epoch 8/70
429/429 [==============================] - 0s 593us/step - loss: 0.0044
Epoch 9/70
429/429 [==============================] - 0s 601us/step - loss: 0.0037
Epoch 10/70
429/429 [==============================] - 0s 590us/step - loss: 0.0034
Epoch 11/70
429/429 [==============================] - 0s 597us/step - loss: 0.0034
Epoch 12/70
429/429 [==============================] - 0s 676us/step - loss: 0.0033
E

In [132]:
from sklearn.metrics import mean_squared_error

# 테스트 데이터 정규화
normalized_X_test = scaler_X.transform(X_test)
normalized_y_test = scaler_y.transform(y_test.reshape(-1, 1))

# LSTM에 입력으로 넣기 위한 형태로 변환
X_test_reshaped = normalized_X_test.reshape((normalized_X_test.shape[0], 1, normalized_X_test.shape[1]))

# 모델을 사용하여 테스트 데이터 예측
predicted_scaled = model.predict(X_test_reshaped)

# 예측 결과 역정규화
predicted = scaler_y.inverse_transform(predicted_scaled)

# 실제 값 역정규화
actual = scaler_y.inverse_transform(normalized_y_test)

# MSE 계산
mse = mean_squared_error(actual, predicted)

# 결과 출력
for i in range(len(predicted)):
    print(f"Actual: {actual[i]}, Predicted: {predicted[i][0]}")

# MSE 출력
print(f"MSE: {mse}")


1428/1428 [==============================] - 1s 313us/step
Actual: [1485.38387097], Predicted: 1658.427978515625
Actual: [1485.38387097], Predicted: 1519.0745849609375
Actual: [1278.08602151], Predicted: 1727.4381103515625
Actual: [519.85253456], Predicted: 818.3973999023438
Actual: [1278.08602151], Predicted: 1897.188232421875
Actual: [1485.38387097], Predicted: 1671.697509765625
Actual: [1278.08602151], Predicted: 2002.9866943359375
Actual: [1278.08602151], Predicted: 1710.14892578125
Actual: [1485.38387097], Predicted: 1816.9481201171875
Actual: [239.33064516], Predicted: 157.13633728027344
Actual: [1278.08602151], Predicted: 2015.2318115234375
Actual: [519.85253456], Predicted: 816.0433349609375
Actual: [1485.38387097], Predicted: 1739.6551513671875
Actual: [239.33064516], Predicted: 120.02869415283203
Actual: [239.33064516], Predicted: 219.5025177001953
Actual: [519.85253456], Predicted: 921.2833862304688
Actual: [1485.38387097], Predicted: 1132.6424560546875
Actual: [1278.0860215

In [ ]:
''' 여긴 진짜 test 데이터로 넣어본거
# 새로운 데이터에 대한 예측
# (예측값을 다시 역정규화하여 원래 스케일로 되돌릴 수 있습니다.)
new_data = test
# 'train' 데이터프레임에서 사용한 feature들의 순서를 가져옴
train_feature_order = X_train.columns.tolist()

# 'new_data'에서 사용한 feature들의 순서를 가져옴
new_data_feature_order = new_data.columns.tolist()

# 'new_data'를 'train' 데이터프레임의 feature 순서에 맞게 재정렬
new_data_reordered = new_data[train_feature_order]

# 'scaler_X'를 다시 초기화하여 'new_data_reordered'를 변환
new_data_scaled = scaler_X.transform(new_data_reordered)
# 새로운 데이터도 정규화
new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

predicted_scaled = model.predict(new_data_reshaped)

# 예측 결과 역정규화
predicted = scaler_y.inverse_transform(predicted_scaled)

# 예측 결과 출력
print(predicted)

35/35 [==============================] - 0s 363us/step
[[3228.4966]
 [3313.3066]
 [3398.351 ]
 ...
 [1282.1215]
 [1300.9712]
 [1319.0067]]


In [ ]:

submission = submission = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/sample_submission.csv')
submission['answer'] = predicted
submission.to_csv('submission_lstm_pepero2.csv', index = False)
